In [47]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(project_root)
if project_root not in sys.path:
    sys.path.append(project_root)

c:\Users\Camille\Documents\TWR


In [48]:
from training_models.config.database import AsyncSessionLocal
from training_models.repositories.campaigns_repository import CampaignRepository
from training_models.repositories.request_repository import RequestsRepository
from training_models.config.mongo import mongo_collection, mongo_client

campaign_repository = CampaignRepository(session_factory=AsyncSessionLocal)
mongo_request_repository = RequestsRepository(collection=mongo_collection)

In [49]:
from attention_based import MILTrainingService

In [50]:
# traffic_source_alvo = "tiktok" # Altere para a fonte real

# training_service = MILTrainingService(
#     traffic_source=traffic_source_alvo,
#     repo_requests=mongo_request_repository,
#     repo_campaigns=campaign_repository,
#     device="cuda" if torch.cuda.is_available() else "cpu",
#     emb_config="fasttext",
#     bag_size=3
# )

# # 4. Execução Direta (A mágica do Jupyter)
# try:
#     print(f"Iniciando pipeline para a fonte: {traffic_source_alvo}")
    
#     # É só usar o await direto na célula!
#     await training_service.training_pipeline(
#         epochs=15, 
#         path="models"
#     )
    
# except Exception as e:
#     print(f"Ocorreu um erro durante o pipeline: {e}")
    
# finally:
#     # Fecha as conexões com o banco de dados
#     print("Limpando conexões com o banco de dados...")


In [51]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import numpy as np

def plot_attention_for_bot_ips(modelo, dataloader, device, num_ips_to_plot=3):
    """
    Procura por IPs que o modelo classificou corretamente como Bots (Verdadeiros Positivos)
    e plota o peso de atenção que cada requisição HTTP daquele IP recebeu.
    """
    modelo.eval()
    ips_plotados = 0
    
    with torch.no_grad():
        for i, (bag, label) in enumerate(dataloader):
            bag = bag.to(device)
            label = label.to(device)
            
            # Pula os humanos (queremos ver como ele caça os bots)
            if label.item() == 0.0:
                continue
                
            # Faz a previsão
            pred, attention = modelo(bag)
            
            # Se o modelo acertou que é um bot (Verdadeiro Positivo)
            if pred.item() > 0.5:
                # Extrai os pesos (remove dimensões extras do batch=1)
                pesos = attention.squeeze().cpu().numpy()
                
                # Se o IP tiver apenas 1 requisição, o numpy pode retornar um float em vez de array.
                # Garantimos que seja um array para o plot.
                if pesos.ndim == 0:
                    pesos = np.array([pesos])
                
                num_requisicoes = len(pesos)
                
                # Prepara o gráfico
                plt.figure(figsize=(10, 4))
                
                # Destaca em vermelho a requisição com maior peso (o "gatilho" do bot)
                cores = ['red' if peso == max(pesos) else 'skyblue' for peso in pesos]
                
                plt.bar(range(num_requisicoes), pesos, color=cores, edgecolor='black')
                
                # Configurações visuais
                plt.title(f"Auditoria de IP Malicioso #{ips_plotados + 1}\nPrevisão do Modelo: {pred.item():.4f} (1.0 = Certeza Absoluta de Bot)", fontsize=12)
                plt.xlabel("Índice da Requisição HTTP na Sessão", fontsize=10)
                plt.ylabel("Peso de Atenção (Culpa)", fontsize=10)
                plt.xticks(range(num_requisicoes))
                plt.ylim(0, 1.05) # Atenção vai de 0 a 1
                plt.grid(axis='y', linestyle='--', alpha=0.7)
                
                plt.tight_layout()
                plt.show()
                
                ips_plotados += 1
                
                if ips_plotados >= num_ips_to_plot:
                    break

# Como chamar a função no seu Notebook:

In [52]:
hashes = ["0h5ff0hx9e", "3zdba5h0e9", "9ct9zcf7xe", 'eoqs2p9hvl', "gp0cj2b193", "mdwcdjlo0h"]

In [53]:
import pandas as pd

results, hashes_info = await mongo_request_repository.get_training_sample_by_hashes(
      hashes=hashes, 
      limit_each=500,
      rule_id=True
)
df = pd.DataFrame(results)

[{'_id': 'mdwcdjlo0h', 'start_datetime': datetime.datetime(2026, 2, 18, 17, 4, 15, 726000), 'end_datetime': datetime.datetime(2026, 2, 25, 16, 21, 38, 795000)}, {'_id': '9ct9zcf7xe', 'start_datetime': datetime.datetime(2026, 2, 18, 18, 26, 5, 945000), 'end_datetime': datetime.datetime(2026, 2, 21, 0, 27, 53, 572000)}, {'_id': 'eoqs2p9hvl', 'start_datetime': datetime.datetime(2026, 2, 18, 19, 13, 36, 85000), 'end_datetime': datetime.datetime(2026, 2, 20, 12, 3, 27, 133000)}, {'_id': '0h5ff0hx9e', 'start_datetime': datetime.datetime(2026, 2, 18, 17, 2, 42, 262000), 'end_datetime': datetime.datetime(2026, 2, 25, 17, 3, 22, 974000)}, {'_id': 'gp0cj2b193', 'start_datetime': datetime.datetime(2026, 2, 18, 17, 3, 30, 884000), 'end_datetime': datetime.datetime(2026, 2, 25, 16, 35, 27, 346000)}]


In [54]:
from asyncio.log import logger
from attention_based import MILBagDatasetLogical
from torch.utils.data import DataLoader
from data.embedding_service import EmbeddingService

df["decision"] = df["decision"].str.lower().replace({"bot": "bots"})

mapeamento_mil = {"bots": 1, "unsafe": 0}
df["decision_mil"] = df["decision"].map(mapeamento_mil)

PATH = "G:/Meu Drive/TWR/data"
LABEL_MAP = {"bot": 0, "unsafe": 1, "bots": 0}
conf = "fasttext"
traffic_source = 'tiktok'
model_path_ft = f"{PATH}/{traffic_source}/fasttext_{traffic_source}.model"  
# model_path_ft = "all-MiniLM-L6-v2"

try:
      EmbeddingService.get_instance(conf, model_path_ft, traffic_source=traffic_source)
except FileNotFoundError:
      print(f"Error: Embedding model not found at {model_path_ft}")

embeddings_matrix, texts = EmbeddingService.process_and_encode(df)

embeddings_matrix, texts = EmbeddingService.process_and_encode(df)

df["embedding"] = list(embeddings_matrix)

logger.info(f"Agrupando {len(df)} requisições por IPs únicos...")

bags_df = df.groupby("ip").agg({
      "embedding": list,       
      "decision_mil": list    
}).reset_index()

bags_df["bag_label"] = bags_df["decision_mil"].apply(lambda labels: 1.0 if 1 in labels else 0.0)

dataset_test = MILBagDatasetLogical(bags_df)
test_loader = DataLoader(dataset_test, batch_size=1, shuffle=False)

Enter to Fasttext encoder


Criando Vocabulário: 100%|██████████| 1000/1000 [00:00<00:00, 1149.10it/s]


Using 11 out of 12 cores


Vetorizando: 100%|██████████| 1000/1000 [00:14<00:00, 69.61it/s]


Finishing encoding
Enter to Fasttext encoder


Criando Vocabulário: 100%|██████████| 1000/1000 [00:00<00:00, 1483.50it/s]


Using 11 out of 12 cores


Vetorizando: 100%|██████████| 1000/1000 [00:11<00:00, 83.92it/s]

Finishing encoding
2026-02-25 14:03:57,170 - INFO - Agrupando 1000 requisições por IPs únicos...


In [55]:
from attention_based import AttentionMIL

modelo = AttentionMIL(in_features=300, hidden_dim=256)
checkpoint = torch.load("models/tiktok/fasttext/attention_mil.pth", weights_only=False)
pesos = checkpoint["model_state_dict"]

modelo.load_state_dict(pesos)

modelo.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [56]:
# plot_attention_with_ip(modelo, dataloader=test_loader, device=device)

In [57]:
import json

import torch
import numpy as np
import pandas as pd

def cacar_bots_camuflados(modelo, dataloader, df_original, device, threshold=0.5, limite_exibicao=5):
    """
    Procura IPs que estão rotulados como 'unsafe' (0.0) no banco de dados,
    mas que o algoritmo MIL classificou como 'bot' (1.0) com alta certeza.
    """
    modelo.eval()
    bots_camuflados_encontrados = 0
    
    print("🕵️‍♂️ Iniciando caçada aos Bots Camuflados...")
    print(f"Buscando IPs rotulados como Humanos, mas previstos como Bot (> {threshold*100}%)\n")
    print("=" * 60)
    
    with torch.no_grad():
        for bag, label, ip_tuple in dataloader:
            
            # 1. Filtro do Banco de Dados: Só queremos olhar para quem foi rotulado como Humano (0.0)
            if label.item() != 0.0:
                continue
                
            bag = bag.to(device)
            pred, attention = modelo(bag)
            
            # 2. Filtro do Modelo: O modelo discorda do banco e tem certeza de que é um Bot?
            if pred.item() > threshold:
                ip_string = ip_tuple[0]
                bots_camuflados_encontrados += 1
                
                print(f"🚨 BOT CAMUFLADO DETECTADO | IP: {ip_string}")
                print(f"   Certeza da IA: {pred.item() * 100:.2f}%")
                
                # 3. Busca o Histórico no DataFrame Original
                # Pega todas as linhas originais desse IP
                logs_do_ip = df_original[df_original['ip'] == ip_string].reset_index(drop=True)
                print(len(logs_do_ip))
                
                # Extrai os pesos de atenção para sabermos qual requisição entregou o disfarce
                pesos = attention.squeeze().cpu().numpy()
                if pesos.ndim == 0:
                    pesos = np.array([pesos])
                
                # Encontra o índice da requisição que recebeu a maior nota de culpa
                idx_maior_peso = np.argmax(pesos)
                
                print(f"   Total de Requisições na Sessão: {len(logs_do_ip)}")
                print(f"   Requisição que desmascarou o Bot (Peso: {pesos[idx_maior_peso]:.4f}):")
                
                # 4. Imprime o Texto Original (Request/Headers)
                linha_culpada = logs_do_ip.iloc[idx_maior_peso]
                
                # Exibe a URL acessada e os Headers
                print(f"   -> Headers: ")
                json_header = json.loads(linha_culpada.get('headers', 'Coluna request não encontrada'))
                print(json.dumps(json_header, indent=4))
                # Se você tiver a coluna headers como string/dict, podemos imprimi-la também:
                # print(f"   -> Headers: {linha_culpada.get('headers', 'N/A')}")
                
                print("-" * 60)
                
                if bots_camuflados_encontrados >= limite_exibicao:
                    print(f"Parando a exibição nos primeiros {limite_exibicao} bots encontrados.")
                    break
                    
    if bots_camuflados_encontrados == 0:
        print("Nenhum bot camuflado encontrado neste lote de teste! O seu banco de dados está bem limpo.")

In [58]:
# Chama a função para inspecionar os 10 primeiros bots camuflados que o modelo encontrar
cacar_bots_camuflados(
    modelo=modelo, 
    dataloader=test_loader, 
    df_original=df, 
    device=device, 
    threshold=0.5, 
    limite_exibicao=10
)

🕵️‍♂️ Iniciando caçada aos Bots Camuflados...
Buscando IPs rotulados como Humanos, mas previstos como Bot (> 50.0%)

🚨 BOT CAMUFLADO DETECTADO | IP: 2600:387:f:6d12::5
   Certeza da IA: 54.01%
1
   Total de Requisições na Sessão: 1
   Requisição que desmascarou o Bot (Peso: 1.0000):
   -> Headers: 
{
    "host": "twr.vitalityuniversal.com",
    "sec-fetch-mode": "navigate",
    "accept-encoding": "gzip, br",
    "sec-fetch-dest": "document",
    "accept-language": "en-US,en",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/29.0 Chrome/136.0.0.0 Mobile Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "x-request-id": "e5a205c9bc07ecf139ad0b251a7b575f",
    "x-real-ip": "2600:387:f:6d12::5",
    "x-forwarded-for": "2600:387:f:6d12::5",
    "x-forwarded-host": "twr.vitali

## Testando com ids falsos

In [59]:
df["rule_id_list"].value_counts()

rule_id_list
[]                                      500
[1170]                                  477
[1170, 1181]                             10
[1170, 1183]                              5
[1170, 1172, 1183]                        1
[1172]                                    1
[1196]                                    1
[1170, 1180, 1183, 1189]                  1
[1165, 1170, 1181, 1198]                  1
[1175, 1176, 1203]                        1
[1169, 1175, 1176, 1188, 1201, 1203]      1
[1169, 1175, 1176, 1183, 1185, 1201]      1
Name: count, dtype: int64

In [60]:
idx_com_1170 = df[
    df["rule_id_list"].apply(lambda x: isinstance(x, list) and 1175 in x)
].index

percentual = 0.3

idx_sorteados = idx_com_1170.to_series().sample(
    frac=percentual,
    random_state=42 
).index

In [61]:
df.loc[idx_sorteados, "decision"] = "unsafe"
df.loc[idx_sorteados, ["decision", "rule_id_list"]].head()

,decision,rule_id_list
323,unsafe,"[1175, 1176, 1203]"


In [62]:
import numpy as np

df["changed"] = False
df.loc[idx_sorteados, "changed"] = True
df["changed"].value_counts()

changed
False    999
True       1
Name: count, dtype: int64

In [63]:
bags_df_test = df.groupby("ip").agg({
    "embedding": list,
    "decision_mil": list,
    "changed": list,
    "headers": list
}).reset_index()

bags_df_test["tem_isca"] = bags_df_test["changed"].apply(lambda lista: True in lista)

print(f"Total de IPs agrupados: {len(bags_df_test)}")
print(f"Total de IPs com iscas (modificados por você): {bags_df_test['tem_isca'].sum()}")

Total de IPs agrupados: 959
Total de IPs com iscas (modificados por você): 1


In [64]:
bags_df_test[bags_df_test["tem_isca"] == True]["decision_mil"].value_counts()

decision_mil
[1]    1
Name: count, dtype: int64

In [65]:
filtro_mistas = bags_df_test["decision_mil"].apply(lambda lista: (1 in lista) and (0 in lista))
bags_mistas = bags_df_test[filtro_mistas]
display(bags_mistas[["ip", "decision_mil", "headers"]])

,ip,decision_mil,headers
324,2600:387:15:5f14::a,"[1, 1, 0]","[{""host"":""twr.vitalityuniversal.com"",""referer""..."
346,2600:387:5:805::ae,"[1, 1, 1, 1, 0]","[{""host"":""twr.vitalityuniversal.com"",""referer""..."
541,2601:845:8100:a820:d66:22a5:46e7:7bfa,"[1, 0]","[{""host"":""twr.vitalityuniversal.com"",""referer""..."
635,2603:9008:1200:98ab:1506:b064:c4c4:5080,"[1, 1, 0]","[{""host"":""twr.vitalityuniversal.com"",""referer""..."


In [66]:
analise = bags_mistas.iloc[0]["headers"]

json_analise = json.loads(analise[1])
print(json.dumps(json_analise, indent=4))

{
    "host": "twr.vitalityuniversal.com",
    "referer": "https://www.tiktok.com/",
    "accept-encoding": "gzip, br",
    "user-agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 18_7 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 musical_ly_43.8.0 JsSdk/2.0 NetType/2G Channel/App Store ByteLocale/en Region/US isDarkMode/0 WKWebView/1 RevealType/Dialog",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "x-request-id": "32eac463aeb18a746aadb10c8533eddc",
    "x-real-ip": "2600:387:15:5f14::a",
    "x-forwarded-for": "2600:387:15:5f14::a",
    "x-forwarded-host": "twr.vitalityuniversal.com",
    "x-forwarded-port": "80",
    "x-forwarded-proto": "http",
    "x-forwarded-scheme": "http",
    "x-scheme": "http",
    "x-original-forwarded-for": "2600:387:15:5f14::a, 104.23.254.115",
    "forwarded": "for=104.23.254.115",
    "x-tt-trace-id": "00-95b29de11066a55fcad285462a6304d1-95b29de11066a55f-01",
    "x-khronos": "1772038036",
    "x-

In [67]:
json_analise = json.loads(analise[0])
print(json.dumps(json_analise, indent=4))

{
    "host": "twr.vitalityuniversal.com",
    "referer": "https://www.tiktok.com/",
    "accept-encoding": "gzip, br",
    "user-agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 18_7 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 musical_ly_43.8.0 JsSdk/2.0 NetType/2G Channel/App Store ByteLocale/en Region/US isDarkMode/0 WKWebView/1 RevealType/Dialog",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "x-request-id": "6691a7a444337657792d04bddc992184",
    "x-real-ip": "2600:387:15:5f14::a",
    "x-forwarded-for": "2600:387:15:5f14::a",
    "x-forwarded-host": "twr.vitalityuniversal.com",
    "x-forwarded-port": "80",
    "x-forwarded-proto": "http",
    "x-forwarded-scheme": "http",
    "x-scheme": "http",
    "x-original-forwarded-for": "2600:387:15:5f14::a, 104.23.254.170",
    "forwarded": "for=104.23.254.170",
    "x-tt-trace-id": "00-95b8101d1066a55fcad285462a5804d1-95b8101d1066a55f-01",
    "x-khronos": "1772038393",
    "x-

In [ ]:
import torch
import numpy as np

def auditar_injecao_adversarial(modelo, dataloader, bags_df_test, device):
    modelo.eval()
    acertos_bag = 0
    acertos_atencao = 0
    total_iscas = 0
    
    print("🎯 Iniciando Auditoria de Teste Adversarial...")
    print("Verificando se o modelo encontra as suas modificações...\n" + "="*60)
    
    with torch.no_grad():
        for bag, label, ip_tuple in dataloader:
            ip_string = ip_tuple[0]
            
            info_ip = bags_df_test[bags_df_test['ip'] == ip_string].iloc[0]
            
            if not info_ip['tem_isca']:
                continue
                
            total_iscas += 1
            lista_changed = info_ip['changed'] 
            
            bag = bag.to(device)
            pred, attention = modelo(bag)
            certeza = pred.item()
            
            print(f"\n🕵️ Investigando IP Injetado: {ip_string} | Total de Reqs na Sessão: {len(lista_changed)}")

            if certeza > 0.5:
                print(f"  [1. BAG] ✅ SUCESSO: Modelo cravou como BOT com {certeza*100:.2f}% de certeza.")
                acertos_bag += 1
            else:
                print(f"  [1. BAG] ❌ FALHA: O modelo achou que era humano ({certeza*100:.2f}%). O seu disfarce foi perfeito.")
                

            pesos = attention.squeeze().cpu().numpy()
            if pesos.ndim == 0: pesos = np.array([pesos])
            
            idx_maior_peso = np.argmax(pesos)
            
            req_estava_modificada = lista_changed[idx_maior_peso]
            
            if req_estava_modificada:
                print(f"  [2. ATENÇÃO] 🎯 SUCESSO: O modelo apontou exatamente para o índice {idx_maior_peso}, que foi o que você adulterou!")
                acertos_atencao += 1
            else:
                print(f"  [2. ATENÇÃO] ⚠️ AVISO: O modelo focou no índice {idx_maior_peso} (Peso: {pesos[idx_maior_peso]:.2f}), mas você tinha alterado outro índice. Ele se confundiu ou achou outro bot real.")
                
    print("\n" + "="*60)
    print("📊 RESUMO FINAL DA AUDITORIA:")
    print(f"Total de IPs Adulterados Testados: {total_iscas}")
    if total_iscas > 0:
        print(f"Taxa de Captura do IP (Bag): {acertos_bag/total_iscas * 100:.1f}%")
        print(f"Taxa de Precisão da Atenção (Na Mosca): {acertos_atencao/total_iscas * 100:.1f}%")

auditar_injecao_adversarial(modelo, test_loader, bags_df_test, device)

🎯 Iniciando Auditoria de Teste Adversarial...
Verificando se o modelo encontra as suas modificações...

🕵️ Investigando IP Injetado: 2600:1002:b163:82a6:8437:cba5:7f67:38b5 | Total de Reqs na Sessão: 1
  [1. BAG] ✅ SUCESSO: Modelo cravou como BOT com 95.73% de certeza.
  [2. ATENÇÃO] 🎯 SUCESSO: O modelo apontou exatamente para o índice 0, que foi o que você adulterou!

📊 RESUMO FINAL DA AUDITORIA:
Total de IPs Adulterados Testados: 1
Taxa de Captura do IP (Bag): 100.0%
Taxa de Precisão da Atenção (Na Mosca): 100.0%
